In [1]:
import yasa
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import decimate
import mne
import os
import glob

In [2]:
from initial_processes import *

In [3]:
def load_32_EEG_by_state(foldername, montage_name, source_number, brain_states, brain_state, downsampling):

    'Below are 2 functions from OpenEphys to load data channels and auxilary (accelerometer) channels'
    data=loadFolderToArray(foldername, channels = 'all', chprefix = 'CH', dtype = float, session = '0', source = source_number)
    data_aux=loadFolderToArray(foldername, channels = 'all', chprefix = 'AUX', dtype = float, session = '0', source = source_number)

    #data=loadFolderToArray(prm.get_filepath(), channels = 'all', chprefix = 'CH', dtype = float, session = '0', source = '101')
    #data_aux=loadFolderToArray(prm.get_filepath(), channels = 'all', chprefix = 'AUX', dtype = float, session = '0', source = '101')

    'Below we append a line to the data array and add the accelrometer data. We transpose to fit the MNE data format.'
    data = np.append(data, (np.zeros((data.shape[0],1), dtype=int64)), axis=1)
    #data[:,32]=data_aux[:,0]*800 # zeros

    # As the bins are for every 5 seconds, we need to create an array repeating those states per each ms*downsampling
    brain_states_ms = np.repeat(brain_states, 5000/downsampling)
    
    # Downsampling
    raw_data_downsampled = []
    for i in np.arange(33):
        raw_data_downsampled.append(decimate(data[:,i], downsampling))    
    
    # 34d array. First for states, then for the electrodes
    state_voltage_list = [brain_states_ms]
    size_brain_states = np.size(brain_states_ms)
    for i in np.arange(33):
        state_voltage_list.append(raw_data_downsampled[i][0:size_brain_states])   
    
    state_voltage_array = np.stack(state_voltage_list)
    
    # Amplitude filter
    amp_filter = 200
    state_voltage_array = state_voltage_array[:, abs(state_voltage_array[2,:]) < amp_filter]
    
    # Only takes the voltages with a certain brain state
    volt_by_state = state_voltage_array[1:, state_voltage_array[0,:] == brain_state]
    
    #yasa expect volts, but we are recording in uV
    datatp=volt_by_state #Array from openephys has to be transposed to match RawArray MNE function to create.
    
    del data
    
    
    if isinstance('montage_name', str):
        montage = mne.channels.read_custom_montage(montage_name)
    else:
        print("The montage name is not valid")

    channel_types=['eeg','eeg','eeg','eeg','eeg','eeg', 'eeg', 'eeg',
                   'eeg','eeg','eeg','eeg','eeg','eeg','eeg','eeg'
                   ,'eeg','eeg','eeg','eeg','eeg','eeg','eeg','eeg'
                   ,'eeg','eeg','eeg','eeg','eeg','eeg','eeg','eeg', 'emg'] # not necessario in new version, 'emg', 'emg', 'emg']

    info = mne.create_info(montage.ch_names, prm.get_sampling_rate(), ch_types=channel_types)

    'This makes the object that contains all the data and info about the channels.'
    'Computations like plotting, averaging, power spectrums can be performed on this object'

    custom_raw = mne.io.RawArray( datatp, info)
    del datatp
    return custom_raw

In [4]:
def electrode_combinations(montage_name, neighbors_dist, long_distance, n_elect = 32):
  montage = mne.channels.read_custom_montage(montage_name)
  electrode_names = montage.ch_names[0:n_elect]
  electrode_pos = np.zeros((32,2))
  for i in np.arange(32):
    electrode_pos[i] = montage.dig[i+3].get('r')[0:2].round(3)*50 # in mm

  distances_btw_electrodes = spatial.distance.pdist(electrode_pos, 'euclidean')

  nums = np.linspace(0, n_elect-1, n_elect, dtype = int)

  comb = combinations(nums, 2)
  # working with the combination element is difficult and it can only be assigned once -> it is transformed into a list
  comb_long_distance = list(comb)

  comb = combinations(nums, 2)
  comb_short_distance = list(comb)

  # indexes for the elements to delete
  indexes_to_delete_in_short_distance = []
  indexes_to_delete_in_long_distance = []

  nei = 0
  s_d = 0
  l_d = 0

  for i in range(len(distances_btw_electrodes)):
      if distances_btw_electrodes[i] <= neighbors_dist:
          indexes_to_delete_in_long_distance.append(i)
          indexes_to_delete_in_short_distance.append(i)
          nei += 1
      elif distances_btw_electrodes[i] <long_distance:
          indexes_to_delete_in_long_distance.append(i)
          s_d += 1
      else:
          indexes_to_delete_in_short_distance.append(i)
          l_d += 1

  long_dist_electrodes = np.delete(distances_btw_electrodes, indexes_to_delete_in_long_distance)
  short_dist_electrodes = np.delete(distances_btw_electrodes, indexes_to_delete_in_short_distance)

  # when it deletes an element, it is necessary to update the indexes substracting one to the total of them the list has.
  indexes_already_del = 0
  for i in range(len(indexes_to_delete_in_long_distance)):
      del comb_long_distance[indexes_to_delete_in_long_distance[i] - indexes_already_del]
      indexes_already_del += 1

  indexes_already_del = 0
  for i in range(len(indexes_to_delete_in_short_distance)):
      del comb_short_distance[indexes_to_delete_in_short_distance[i] - indexes_already_del]
      indexes_already_del += 1

  return comb_short_distance, comb_long_distance

In [5]:
ko_folder = '/media/jorge/DATADRIVE0/Data/Alfredo/Syngap/KO/'
wt_folder = '/media/jorge/DATADRIVE0/Data/Alfredo/Syngap/WT/'

In [6]:
montage_name = '/media/jorge/DATADRIVE0/Code/MNE_Alfredo/standard_32grid_Alfredo.elc'
n_electrodes = 32
neighbors_dist = 1
long_distance = 2

prm.set_sampling_rate(1000)
sr=prm.get_sampling_rate()
downsampling = 8
brain_state = 1 # NonREM

### Get the files and folders to analyse

In [10]:
l_folders_KO = []
l_xls_files_KO = []
l_folders_WT = []
l_xls_files_WT = []

# dividing files by animal (KO)
os.chdir(ko_folder)
d = os.getcwd() + '/'
matching_files = glob.glob(r'*xls')
for matching_file in matching_files:
    l_xls_files_KO.append(d+matching_file)
    l_folders_KO.append(d+ matching_file.replace('_States.xls', '') + '/')    

# dividing files by animal (WT)
os.chdir(wt_folder)
d = os.getcwd() + '/'
matching_files = glob.glob(r'*xls')
for matching_file in matching_files:
    l_xls_files_WT.append(d+matching_file)
    l_folders_WT.append(d+matching_file.replace('_States.xls', '') + '/')

### Calculating the combinations for short and long distance

In [7]:
short_d_comb, long_d_comb = electrode_combinations(montage_name, neighbors_dist, long_distance, n_electrodes)

### Calculate the spindles happening together when comparing combinations of electrodes at a short or long distance

In [ ]:
list_sp_multi_WT = []
list_sp_WT = []
sp_short_WT = []
for i, xls_file in enumerate(l_xls_files_WT):
    # First extract data for the whole recording but just for the NonREM state
    brain_states = import_brain_states(xls_file)
    raw_data = load_32_EEG_by_state(l_folders_WT[i], montage_name, '100', brain_states, brain_state, downsampling)
    # Go through all the combinations on that recording
    # Short distances
    for i in list(short_d_comb): 
        data = np.vstack((raw_data._data[i[0]], raw_data._data[i[1]]))
        names = [raw_data.ch_names[i[0]],raw_data.ch_names[i[1]]]
        sp = yasa.spindles_detect(data, sf = sr/downsampling, ch_names = names, multi_only=True, remove_outliers=True)
        sp_short_WT.append(sp)
    # Long distance
    print('recording done:')
    print(xls_file)

 Openning Excelfile /media/jorge/DATADRIVE0/Data/Alfredo/Syngap/WT/S7031_D2-A_States.xls
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Avg. Load Time: 1.6843930631875992 sec
Total Lo

13-Dec-20 22:39:15 | WARNING | No spindle were found in channel V2L_V1B_LEFT.
13-Dec-20 22:39:39 | WARNING | No spindle were found in channel V2MM_RIGHT.


recording done:
/media/jorge/DATADRIVE0/Data/Alfredo/Syngap/WT/S7031_D2-A_States.xls
 Openning Excelfile /media/jorge/DATADRIVE0/Data/Alfredo/Syngap/WT/S7018_D5_States.xls
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous

### The same for KO

In [ ]:
list_sp_multi_KO = []
list_sp_KO = []
sp_short_KO = []
for i, xls_file in enumerate(l_xls_files_KO):
    # First extract data for the whole recording but just for the NonREM state
    brain_states = import_brain_states(xls_file)
    raw_data = load_32_EEG_by_state(l_folders_KO[i], montage_name, '100', brain_states, brain_state, downsampling)
    # Go through all the combinations on that recording
    # Short distances
    for i in list(short_d_comb): 
        data = np.vstack((raw_data._data[i[0]], raw_data._data[i[1]]))
        names = [raw_data.ch_names[i[0]],raw_data.ch_names[i[1]]]
        sp = yasa.spindles_detect(data, sf = sr/downsampling, ch_names = names, multi_only=True, remove_outliers=True)
        sp_short_KO.append(sp)
    # Long distance. do not do it for now, it would take too long
    print('recording done:')
    print(xls_file)

### More analysis here

In [37]:
list_sp_multi_WT = []
list_sp_WT = []
for i, xls_file in enumerate(l_xls_files_WT):
    print('spindles for recording', l_folders_WT[i])
    brain_states = import_brain_states(xls_file)
    raw_data = load_32_EEG_by_state(l_folders_WT[i], montage_name, '100', brain_states, brain_state, downsampling)
    sp = yasa.spindles_detect(raw_data, multi_only=False)
    sp_multi = yasa.spindles_detect(raw_data, multi_only=True, remove_outliers=True)
    list_sp_WT.append(sp)
    list_sp_multi_WT.append(sp_multi)
    print('recording done:')
    print(xls_file)

spindles for recording /media/jorge/DATADRIVE0/Data/Alfredo/Syngap/WT/S7031_D2-A/
 Openning Excelfile /media/jorge/DATADRIVE0/Data/Alfredo/Syngap/WT/S7031_D2-A_States.xls
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous 

12-Dec-20 22:27:51 | ERROR | Wrong data amplitude for Aux (trimmed STD = 0.000). Unit of data MUST be uV! Channel will be skipped.
12-Dec-20 22:28:06 | ERROR | Wrong data amplitude for Aux (trimmed STD = 0.000). Unit of data MUST be uV! Channel will be skipped.


recording done:
/media/jorge/DATADRIVE0/Data/Alfredo/Syngap/WT/S7031_D2-A_States.xls
spindles for recording /media/jorge/DATADRIVE0/Data/Alfredo/Syngap/WT/S7018_D5/
 Openning Excelfile /media/jorge/DATADRIVE0/Data/Alfredo/Syngap/WT/S7018_D5_States.xls
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous 

12-Dec-20 22:31:30 | ERROR | Wrong data amplitude for Aux (trimmed STD = 0.000). Unit of data MUST be uV! Channel will be skipped.
12-Dec-20 22:32:00 | ERROR | Wrong data amplitude for Aux (trimmed STD = 0.000). Unit of data MUST be uV! Channel will be skipped.


recording done:
/media/jorge/DATADRIVE0/Data/Alfredo/Syngap/WT/S7018_D5_States.xls
spindles for recording /media/jorge/DATADRIVE0/Data/Alfredo/Syngap/WT/S7018_D6/
 Openning Excelfile /media/jorge/DATADRIVE0/Data/Alfredo/Syngap/WT/S7018_D6_States.xls
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous da

...

recording done:
/media/jorge/DATADRIVE0/Data/Alfredo/Syngap/WT/S7039_D2_States.xls
spindles for recording /media/jorge/DATADRIVE0/Data/Alfredo/Syngap/WT/S7040_D2/
 Openning Excelfile /media/jorge/DATADRIVE0/Data/Alfredo/Syngap/WT/S7040_D2_States.xls
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous da

12-Dec-20 23:18:49 | ERROR | Wrong data amplitude for Aux (trimmed STD = 0.000). Unit of data MUST be uV! Channel will be skipped.
12-Dec-20 23:19:10 | ERROR | Wrong data amplitude for Aux (trimmed STD = 0.000). Unit of data MUST be uV! Channel will be skipped.


recording done:
/media/jorge/DATADRIVE0/Data/Alfredo/Syngap/WT/S7040_D2_States.xls
spindles for recording /media/jorge/DATADRIVE0/Data/Alfredo/Syngap/WT/S7041_D2/
 Openning Excelfile /media/jorge/DATADRIVE0/Data/Alfredo/Syngap/WT/S7041_D2_States.xls
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous da

12-Dec-20 23:21:58 | ERROR | Wrong data amplitude for Aux (trimmed STD = 0.000). Unit of data MUST be uV! Channel will be skipped.
12-Dec-20 23:22:19 | ERROR | Wrong data amplitude for Aux (trimmed STD = 0.000). Unit of data MUST be uV! Channel will be skipped.


recording done:
/media/jorge/DATADRIVE0/Data/Alfredo/Syngap/WT/S7041_D2_States.xls


In [41]:
total_spindles_WT = 0
for record_sp in list_sp_WT:
    total_spindles_WT = total_spindles_WT + record_sp.summary().shape[0]

print('Total number of spindles in WT animals: ', total_spindles_WT)
total_twospindles_WT = 0
for record_sp in list_sp_multi_WT:
    total_twospindles_WT = total_twospindles_WT + record_sp.summary().shape[0]

print('Total number of simultaneous spindles in WT animals: ', total_twospindles_WT)

Total number of spindles in WT animals:  26147
Total number of simultaneous spindles in WT animals:  22120


In [38]:
list_sp_multi_KO = []
list_sp_KO = []
for i, xls_file in enumerate(l_xls_files_KO):
    print('spindles for recording', l_folders_KO[i])
    brain_states = import_brain_states(xls_file)
    raw_data = load_32_EEG_by_state(l_folders_KO[i], montage_name, '100', brain_states, brain_state, downsampling)
    sp = yasa.spindles_detect(raw_data, multi_only=False)
    sp_multi = yasa.spindles_detect(raw_data, multi_only=True, remove_outliers=True)
    list_sp_KO.append(sp)
    list_sp_multi_KO.append(sp_multi)
    print('recording done:')
    print(xls_file)

spindles for recording /media/jorge/DATADRIVE0/Data/Alfredo/Syngap/KO/S7027_D2-B/
 Openning Excelfile /media/jorge/DATADRIVE0/Data/Alfredo/Syngap/KO/S7027_D2-B_States.xls
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous 

12-Dec-20 23:24:09 | ERROR | Wrong data amplitude for Aux (trimmed STD = 0.000). Unit of data MUST be uV! Channel will be skipped.
12-Dec-20 23:24:12 | WARNING | No spindle were found in channel V1M_LEFT.
12-Dec-20 23:24:15 | WARNING | No spindle were found in channel V2L_RIGHT.
12-Dec-20 23:24:16 | ERROR | Wrong data amplitude for Aux (trimmed STD = 0.000). Unit of data MUST be uV! Channel will be skipped.
12-Dec-20 23:24:19 | WARNING | No spindle were found in channel V1M_LEFT.
12-Dec-20 23:24:22 | WARNING | No spindle were found in channel V2L_RIGHT.


recording done:
/media/jorge/DATADRIVE0/Data/Alfredo/Syngap/KO/S7027_D2-B_States.xls
spindles for recording /media/jorge/DATADRIVE0/Data/Alfredo/Syngap/KO/S7017_D5/
 Openning Excelfile /media/jorge/DATADRIVE0/Data/Alfredo/Syngap/KO/S7017_D5_States.xls
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous 

12-Dec-20 23:26:33 | ERROR | Wrong data amplitude for Aux (trimmed STD = 0.000). Unit of data MUST be uV! Channel will be skipped.
12-Dec-20 23:26:57 | ERROR | Wrong data amplitude for Aux (trimmed STD = 0.000). Unit of data MUST be uV! Channel will be skipped.


recording done:
/media/jorge/DATADRIVE0/Data/Alfredo/Syngap/KO/S7017_D5_States.xls
spindles for recording /media/jorge/DATADRIVE0/Data/Alfredo/Syngap/KO/S7017_D6/
 Openning Excelfile /media/jorge/DATADRIVE0/Data/Alfredo/Syngap/KO/S7017_D6_States.xls
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous da

...

recording done:
/media/jorge/DATADRIVE0/Data/Alfredo/Syngap/KO/S7037_D3_States.xls
spindles for recording /media/jorge/DATADRIVE0/Data/Alfredo/Syngap/KO/S7038_D2/
 Openning Excelfile /media/jorge/DATADRIVE0/Data/Alfredo/Syngap/KO/S7038_D2_States.xls
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous da

13-Dec-20 00:26:49 | ERROR | Wrong data amplitude for Aux (trimmed STD = 0.000). Unit of data MUST be uV! Channel will be skipped.
13-Dec-20 00:27:14 | ERROR | Wrong data amplitude for Aux (trimmed STD = 0.000). Unit of data MUST be uV! Channel will be skipped.


recording done:
/media/jorge/DATADRIVE0/Data/Alfredo/Syngap/KO/S7038_D2_States.xls
spindles for recording /media/jorge/DATADRIVE0/Data/Alfredo/Syngap/KO/S7038_D3/
 Openning Excelfile /media/jorge/DATADRIVE0/Data/Alfredo/Syngap/KO/S7038_D3_States.xls
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous data...
Loading continuous da

13-Dec-20 00:30:42 | ERROR | Wrong data amplitude for Aux (trimmed STD = 0.000). Unit of data MUST be uV! Channel will be skipped.
13-Dec-20 00:31:11 | ERROR | Wrong data amplitude for Aux (trimmed STD = 0.000). Unit of data MUST be uV! Channel will be skipped.


recording done:
/media/jorge/DATADRIVE0/Data/Alfredo/Syngap/KO/S7038_D3_States.xls


In [42]:
total_spindles_KO = 0
for record_sp in list_sp_KO:
    total_spindles_KO = total_spindles_KO + record_sp.summary().shape[0]

print('Total number of spindles in KO animals: ', total_spindles_KO)
total_twospindles_KO = 0
for record_sp in list_sp_multi_KO:
    total_twospindles_KO = total_twospindles_KO + record_sp.summary().shape[0]

print('Total number of simultaneous spindles in KO animals: ', total_twospindles_KO)

Total number of spindles in KO animals:  45569
Total number of simultaneous spindles in KO animals:  37276


In [8]:
sp = yasa.spindles_detect(raw_data, multi_only=False)

09-Dec-20 23:57:08 | ERROR | Wrong data amplitude for Aux (trimmed STD = 0.000). Unit of data MUST be uV! Channel will be skipped.
09-Dec-20 23:57:10 | WARNING | No spindle were found in channel V1M_LEFT.
09-Dec-20 23:57:11 | WARNING | No spindle were found in channel M2_FrA_LEFT.
09-Dec-20 23:57:12 | WARNING | No spindle were found in channel V2MM_RIGHT.
09-Dec-20 23:57:12 | WARNING | No spindle were found in channel V2L_RIGHT.


In [9]:
all_spindles = sp.summary()  
all_spindles

,Start,Peak,End,Duration,Amplitude,RMS,AbsPower,RelPower,Frequency,Oscillations,Symmetry,Channel,IdxChannel
0,3.948,5.560,5.875,1.927,365.448255,67.413128,3.466398,0.259940,12.958475,23.0,0.836100,S1Tr_LEFT,0
1,34.805,35.100,35.762,0.957,328.774541,75.104580,3.549150,0.325160,13.728473,11.0,0.307933,S1Tr_LEFT,0
2,50.569,50.730,51.383,0.814,444.288465,100.491532,3.714491,0.282805,13.249235,9.0,0.197546,S1Tr_LEFT,0
3,54.716,54.886,55.558,0.842,405.889481,95.669755,3.834721,0.252123,13.518406,11.0,0.201661,S1Tr_LEFT,0
4,67.491,67.567,68.227,0.736,389.557646,81.245745,3.631573,0.285579,13.559289,9.0,0.103121,S1Tr_LEFT,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,112.346,112.586,113.064,0.718,283.031144,74.515382,3.393089,0.209272,13.537267,7.0,0.333797,S1Tr_RIGHT,31
185,136.816,137.120,137.533,0.717,330.526431,73.461730,3.722845,0.297858,13.602825,10.0,0.423398,S1Tr_RIGHT,31
186,172.283,172.735,173.021,0.738,350.579705,84.285600,3.638894,0.246620,12.992471,9.0,0.611637,S1Tr_RIGHT,31
187,175.027,175.201,175.734,0.707,308.380558,70.179228,3.587822,0.338266,13.228294,8.0,0.245763,S1Tr_RIGHT,31


In [10]:
sp.summary(grp_chan=True, aggfunc='mean')

,Count,Duration,Amplitude,RMS,AbsPower,RelPower,Frequency,Oscillations,Symmetry
Channel,,,,,,,,,
M1_FrA_LEFT,2,0.782000,382.518518,79.034207,3.618538,0.297382,13.339580,9.000000,0.270143
M1_FrA_RIGHT,4,0.717500,328.871778,73.745953,3.601878,0.282087,13.255325,8.750000,0.555977
M1_ant_LEFT,7,0.714714,373.335316,81.576143,3.635194,0.282019,13.413198,8.285714,0.450918
M1_ant_RIGHT,6,0.714333,313.839580,72.319229,3.555553,0.297473,13.208789,8.500000,0.705996
M1_post_LEFT,7,0.725286,422.324350,91.339696,3.779894,0.282617,13.252365,8.142857,0.367980
M1_post_RIGHT,9,0.815556,380.024143,85.547818,3.709794,0.279480,13.222843,9.666667,0.511899
M2_FrA_RIGHT,2,0.721500,369.783418,84.961265,3.792066,0.275334,13.119830,9.000000,0.273175
M2_ant_LEFT,1,0.696000,570.934996,119.782596,3.942653,0.243405,13.446430,9.000000,0.351506
M2_ant_RIGHT,5,0.653200,379.539921,83.881583,3.731704,0.297219,13.309668,7.600000,0.606413


In [12]:
# do not run this one
import ipympl

In [13]:
# do not run this one 
%matplotlib widget
sp.plot_detection();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=0, description='Epoch:', layout=Layout(align_items='center', justify_con…

In [ ]:
# do not run this one
%matplotlib inline
sp.plot_average(center='Peak', time_before=1, time_after=1)

In [9]:
# Detecting when the spindle occur in at least two channels at the same time
sp_multi = yasa.spindles_detect(raw_data, multi_only=True, remove_outliers=True)
all_spindles = sp_multi.summary()
print(all_spindles.shape[0], 'spindles detected that are common to at least two electrodes.')
all_spindles.head().round(3)

11-Dec-20 19:48:36 | ERROR | Wrong data amplitude for Aux (trimmed STD = 0.000). Unit of data MUST be uV! Channel will be skipped.
11-Dec-20 19:48:38 | WARNING | No spindle were found in channel V1M_LEFT.
11-Dec-20 19:48:39 | WARNING | No spindle were found in channel M2_FrA_LEFT.
11-Dec-20 19:48:40 | WARNING | No spindle were found in channel V2MM_RIGHT.
11-Dec-20 19:48:40 | WARNING | No spindle were found in channel V2L_RIGHT.


187 spindles detected that are common to at least two electrodes.


,Start,Peak,End,Duration,Amplitude,RMS,AbsPower,RelPower,Frequency,Oscillations,Symmetry,Channel,IdxChannel
0,3.948,5.560,5.875,1.927,365.448,67.413,3.466,0.260,12.958,23.0,0.836,S1Tr_LEFT,0
1,34.805,35.100,35.762,0.957,328.775,75.105,3.549,0.325,13.728,11.0,0.308,S1Tr_LEFT,0
2,50.569,50.730,51.383,0.814,444.288,100.492,3.714,0.283,13.249,9.0,0.198,S1Tr_LEFT,0
3,54.716,54.886,55.558,0.842,405.889,95.670,3.835,0.252,13.518,11.0,0.202,S1Tr_LEFT,0
4,67.491,67.567,68.227,0.736,389.558,81.246,3.632,0.286,13.559,9.0,0.103,S1Tr_LEFT,0


In [10]:
all_spindles.round(3)

,Start,Peak,End,Duration,Amplitude,RMS,AbsPower,RelPower,Frequency,Oscillations,Symmetry,Channel,IdxChannel
0,3.948,5.560,5.875,1.927,365.448,67.413,3.466,0.260,12.958,23.0,0.836,S1Tr_LEFT,0
1,34.805,35.100,35.762,0.957,328.775,75.105,3.549,0.325,13.728,11.0,0.308,S1Tr_LEFT,0
2,50.569,50.730,51.383,0.814,444.288,100.492,3.714,0.283,13.249,9.0,0.198,S1Tr_LEFT,0
3,54.716,54.886,55.558,0.842,405.889,95.670,3.835,0.252,13.518,11.0,0.202,S1Tr_LEFT,0
4,67.491,67.567,68.227,0.736,389.558,81.246,3.632,0.286,13.559,9.0,0.103,S1Tr_LEFT,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,112.346,112.586,113.064,0.718,283.031,74.515,3.393,0.209,13.537,7.0,0.334,S1Tr_RIGHT,31
183,136.816,137.120,137.533,0.717,330.526,73.462,3.723,0.298,13.603,10.0,0.423,S1Tr_RIGHT,31
184,172.283,172.735,173.021,0.738,350.580,84.286,3.639,0.247,12.992,9.0,0.612,S1Tr_RIGHT,31
185,175.027,175.201,175.734,0.707,308.381,70.179,3.588,0.338,13.228,8.0,0.246,S1Tr_RIGHT,31
